## Problem Statement

In this assignment students need to predict whether a person makes over 50K per year
or not from classic adult dataset using XGBoost.

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None)
test_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test', skiprows = 1, header = None)
col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 
                'occupation','relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week',
                 'native_country', 'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels

In [3]:
train_set.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
train_set.shape

(32561, 15)

## EDA

In [9]:
train_set["isTrain"] = 1

In [10]:
test_set["isTrain"] = 0

In [11]:
com_data= pd.concat([train_set,test_set])

In [12]:
com_data.fillna("unknown",inplace=True)

In [13]:
data=com_data.replace({' ?':"unknown"})

In [15]:
data1=pd.get_dummies(data=data,columns=["workclass","education","marital_status","occupation","relationship","race","sex","native_country"])

In [17]:
data1=data1.replace({' <=50K':0," >50K":1,' <=50K.':0," >50K.":1})

In [18]:
train_set = data1[data1["isTrain"]==1]
test_set =  data1[data1["isTrain"]==0]

train_set.drop("isTrain",axis=1,inplace=True)
test_set.drop("isTrain",axis=1,inplace=True)

/home/skull/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [19]:
target=pd.DataFrame(train_set["wage_class"])
train_set.drop("wage_class",axis=1,inplace=True)

output=pd.DataFrame(test_set["wage_class"])
test_set.drop("wage_class",axis=1,inplace=True)


/home/skull/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


## building model

In [20]:
import xgboost as xgb

In [21]:
dtrain = xgb.DMatrix(train_set.values, target.values)
dtest = xgb.DMatrix(test_set.values)

In [22]:
print("Train dataset contains {0} rows and {1} columns".format(dtrain.num_row(), dtrain.num_col()))
print("Test dataset contains {0} rows and {1} columns".format(dtest.num_row(), dtest.num_col()))

Train dataset contains 32561 rows and 108 columns
Test dataset contains 16281 rows and 108 columns


In [23]:
params = {
    'objective':'binary:logistic',
    'max_depth':2,
    'silent':1,
    'eta':1
}

num_rounds = 5

## Training classifier

In [24]:
bst = xgb.train(params, dtrain, num_rounds)

## predictions

In [25]:
preds_prob = bst.predict(dtest)


In [26]:
out = np.array(output.values)
labels = dtest.get_label()
preds = preds_prob > 0.5 # threshold
correct = 0

for i in range(len(preds)):
    if (out[i] == preds[i]):
        correct += 1

print('Predicted correctly: {0}/{1}'.format(correct, len(preds)))
print('Error: {0:.4f}'.format(1-correct/len(preds)))

Predicted correctly: 13839/16281
Error: 0.1500
